In [2]:
# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

import utils_lr

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.multioutput import MultiOutputRegressor

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [3]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates = ['FECHAHORA'])
datos.head()

,ESTACION,FECHAHORA,ANHO,DIA,MES,HORA,MINUTO,MP1,MP2_5,MP10,...,DIA_SEM,AQI_MP2_5,AQI_MP10,MEDICION_DIA,MP1_ANTERIOR,MP2_5_ANTERIOR,MP10_ANTERIOR,TEMPERATURA_PRONOSTICO,HUMEDAD_PRONOSTICO,PRESION_PRONOSTICO
0,1,2019-12-30 13:00:00,2019,30,12,13,0,2.839,3.989,4.599,...,0,72.0,25.0,157,2.609,3.720,4.519,36.5,42.8,995.1
1,1,2019-12-30 13:05:00,2019,30,12,13,5,9.000,13.079,21.319,...,0,72.0,25.0,158,1.919,2.659,10.340,36.7,42.1,994.9
2,1,2019-12-30 13:10:00,2019,30,12,13,10,1.969,2.369,13.170,...,0,72.0,25.0,159,1.769,2.609,3.419,36.6,43.5,994.8
3,1,2019-12-30 13:15:00,2019,30,12,13,15,1.740,2.379,2.429,...,0,72.0,25.0,160,1.480,2.009,2.159,37.0,41.4,994.7
4,1,2019-12-30 13:20:00,2019,30,12,13,20,2.710,4.119,7.710,...,0,72.0,25.0,161,1.250,1.909,2.809,37.1,40.6,994.6


# 1 dia antes

In [59]:
estacion = 4

daily_aqi = []
prediction_aqi = [] #prediccion = dia anterior

delta = relativedelta(days = 1)

df = datos[datos['ESTACION'] == estacion]

df.reset_index(inplace = True, drop = True)



for i in range(0, df.index.stop, 288):
    
    j = i+287
    
    aqi = df.iloc[j:j+287, :]
    
    
    prediction = df.iloc[i:j, :]
    
    aqi = aqi.AQI_MP2_5
    
    prediction = prediction.AQI_MP2_5
    
    daily_aqi.append(aqi)
        
    prediction_aqi.append(prediction)
    
# y_test = pd.concat(daily_aqi)
# y_pred = pd.concat(prediction_aqi)

In [75]:
print(len(daily_aqi))
print(len(prediction_aqi))

mae = []
rmse = []
mape = []
media = []
for i in range(0, len(daily_aqi)-2):
    mae.append(mean_absolute_error(daily_aqi[i], prediction_aqi[i]))
    mape.append(mean_absolute_percentage_error(daily_aqi[i], prediction_aqi[i]))
    rmse.append(mean_squared_error(daily_aqi[i], prediction_aqi[i], squared = False))
    media.append(daily_aqi[i].mean())
    
print(np.mean(mae))
print(np.mean(mape))
print(np.mean(rmse))
print(np.mean(media))

458
458
13.114799193104712
0.55271082857536
15.071445638937831
30.09471086252216


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131617 entries, 0 to 131616
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   ESTACION                131617 non-null  int64         
 1   FECHAHORA               131617 non-null  datetime64[ns]
 2   ANHO                    131617 non-null  int64         
 3   DIA                     131617 non-null  int64         
 4   MES                     131617 non-null  int64         
 5   HORA                    131617 non-null  int64         
 6   MINUTO                  131617 non-null  int64         
 7   MP1                     131617 non-null  float64       
 8   MP2_5                   131617 non-null  float64       
 9   MP10                    131617 non-null  float64       
 10  TEMPERATURA             131617 non-null  float64       
 11  HUMEDAD                 131617 non-null  float64       
 12  PRESION                 131617

# 7 dias antes (Promedio)

In [132]:
primera_fecha = df.FECHAHORA.min() + relativedelta(days = 7)

daily_aqi = []

minute_aqi = 0
y_pred = {}

primer_index = df[df['FECHAHORA'] == primera_fecha].index[0]

index_1 = 0

minutos = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
bandera = 0

aqi_aux = 0

sumador = 0

for i in range(primer_index, df.index.stop, 288):
    j = i+288
    
    aqi_real = np.asarray(df.iloc[i:j, :].AQI_MP2_5)
    
    daily_aqi.append(aqi_real)
    
    aqi_7_dias = df.iloc[index_1:i, :]
    
    prediction_aqi = []
    
    for k in range(0, 24):
        
        for m in minutos:
            
            
            aqi_momento = aqi_7_dias[(aqi_7_dias['HORA'] == k) & (aqi_7_dias['MINUTO'] == m)]
            
            for idx in aqi_momento.index:
                
                aqi_aux = aqi_aux + aqi_momento.loc[idx, 'AQI_MP2_5']
                bandera = bandera + 1
                
                
            if bandera > 0:

                minute_aqi = aqi_aux / bandera
                #print(minute_aqi)
                
                aqi_aux = 0
                bandera = 0
                
                
                prediction_aqi.append(minute_aqi)
    
    
    y_pred[sumador] = prediction_aqi
    sumador = sumador + 1
    index_1 = index_1 + 288
    
    

    

    

In [136]:
print(len(daily_aqi))
print(len(y_pred))

mae = []
rmse = []
mape = []
media = []
for i in range(0, len(daily_aqi)-2):
    mae.append(mean_absolute_error(daily_aqi[i], y_pred[i]))
    mape.append(mean_absolute_percentage_error(daily_aqi[i], y_pred[i]))
    rmse.append(mean_squared_error(daily_aqi[i], y_pred[i], squared = False))
    media.append(daily_aqi[i].mean())
    
print(np.mean(mae))
print(np.mean(mape))
print(np.mean(rmse))
print(np.mean(media))

451
451
12.568168460777034
0.5396442640941883
13.609436095503536
29.842690546894335


# Promedio 15 dias

In [137]:
primera_fecha = df.FECHAHORA.min() + relativedelta(days = 14)

daily_aqi = []

minute_aqi = 0
y_pred = {}

primer_index = df[df['FECHAHORA'] == primera_fecha].index[0]

index_1 = 0

minutos = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
bandera = 0

aqi_aux = 0

sumador = 0

for i in range(primer_index, df.index.stop, 288):
    j = i+288
    
    aqi_real = np.asarray(df.iloc[i:j, :].AQI_MP2_5)
    
    daily_aqi.append(aqi_real)
    
    aqi_7_dias = df.iloc[index_1:i, :]
    
    prediction_aqi = []
    
    for k in range(0, 24):
        
        for m in minutos:
            
            
            aqi_momento = aqi_7_dias[(aqi_7_dias['HORA'] == k) & (aqi_7_dias['MINUTO'] == m)]
            
            for idx in aqi_momento.index:
                
                aqi_aux = aqi_aux + aqi_momento.loc[idx, 'AQI_MP2_5']
                bandera = bandera + 1
                
                
            if bandera > 0:

                minute_aqi = aqi_aux / bandera
                #print(minute_aqi)
                
                aqi_aux = 0
                bandera = 0
                
                
                prediction_aqi.append(minute_aqi)
    
    
    y_pred[sumador] = prediction_aqi
    sumador = sumador + 1
    index_1 = index_1 + 288
    

print(len(daily_aqi))
print(len(y_pred))

mae = []
rmse = []
mape = []
media = []
for i in range(0, len(daily_aqi)-2):
    mae.append(mean_absolute_error(daily_aqi[i], y_pred[i]))
    mape.append(mean_absolute_percentage_error(daily_aqi[i], y_pred[i]))
    rmse.append(mean_squared_error(daily_aqi[i], y_pred[i], squared = False))
    media.append(daily_aqi[i].mean())
    
print(np.mean(mae))
print(np.mean(mape))
print(np.mean(rmse))
print(np.mean(media))

444
444
12.469318977591035
0.5527010746365587
13.497786764725358
29.435237556561084


In [138]:
primera_fecha = df.FECHAHORA.min() + relativedelta(days = 30)

daily_aqi = []

minute_aqi = 0
y_pred = {}

primer_index = df[df['FECHAHORA'] == primera_fecha].index[0]

index_1 = 0

minutos = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
bandera = 0

aqi_aux = 0

sumador = 0

for i in range(primer_index, df.index.stop, 288):
    j = i+288
    
    aqi_real = np.asarray(df.iloc[i:j, :].AQI_MP2_5)
    
    daily_aqi.append(aqi_real)
    
    aqi_7_dias = df.iloc[index_1:i, :]
    
    prediction_aqi = []
    
    for k in range(0, 24):
        
        for m in minutos:
            
            
            aqi_momento = aqi_7_dias[(aqi_7_dias['HORA'] == k) & (aqi_7_dias['MINUTO'] == m)]
            
            for idx in aqi_momento.index:
                
                aqi_aux = aqi_aux + aqi_momento.loc[idx, 'AQI_MP2_5']
                bandera = bandera + 1
                
                
            if bandera > 0:

                minute_aqi = aqi_aux / bandera
                #print(minute_aqi)
                
                aqi_aux = 0
                bandera = 0
                
                
                prediction_aqi.append(minute_aqi)
    
    
    y_pred[sumador] = prediction_aqi
    sumador = sumador + 1
    index_1 = index_1 + 288
    

print(len(daily_aqi))
print(len(y_pred))

mae = []
rmse = []
mape = []
media = []
for i in range(0, len(daily_aqi)-2):
    mae.append(mean_absolute_error(daily_aqi[i], y_pred[i]))
    mape.append(mean_absolute_percentage_error(daily_aqi[i], y_pred[i]))
    rmse.append(mean_squared_error(daily_aqi[i], y_pred[i], squared = False))
    media.append(daily_aqi[i].mean())
    
print(np.mean(mae))
print(np.mean(mape))
print(np.mean(rmse))
print(np.mean(media))

428
428
12.55132259607025
0.5856092474427101
13.568865466860709
28.981228808033386


In [139]:
primera_fecha = df.FECHAHORA.min() + relativedelta(days = 2)

daily_aqi = []

minute_aqi = 0
y_pred = {}

primer_index = df[df['FECHAHORA'] == primera_fecha].index[0]

index_1 = 0

minutos = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
bandera = 0

aqi_aux = 0

sumador = 0

for i in range(primer_index, df.index.stop, 288):
    j = i+288
    
    aqi_real = np.asarray(df.iloc[i:j, :].AQI_MP2_5)
    
    daily_aqi.append(aqi_real)
    
    aqi_7_dias = df.iloc[index_1:i, :]
    
    prediction_aqi = []
    
    for k in range(0, 24):
        
        for m in minutos:
            
            
            aqi_momento = aqi_7_dias[(aqi_7_dias['HORA'] == k) & (aqi_7_dias['MINUTO'] == m)]
            
            for idx in aqi_momento.index:
                
                aqi_aux = aqi_aux + aqi_momento.loc[idx, 'AQI_MP2_5']
                bandera = bandera + 1
                
                
            if bandera > 0:

                minute_aqi = aqi_aux / bandera
                #print(minute_aqi)
                
                aqi_aux = 0
                bandera = 0
                
                
                prediction_aqi.append(minute_aqi)
    
    
    y_pred[sumador] = prediction_aqi
    sumador = sumador + 1
    index_1 = index_1 + 288
    

print(len(daily_aqi))
print(len(y_pred))

mae = []
rmse = []
mape = []
media = []
for i in range(0, len(daily_aqi)-2):
    mae.append(mean_absolute_error(daily_aqi[i], y_pred[i]))
    mape.append(mean_absolute_percentage_error(daily_aqi[i], y_pred[i]))
    rmse.append(mean_squared_error(daily_aqi[i], y_pred[i], squared = False))
    media.append(daily_aqi[i].mean())
    
print(np.mean(mae))
print(np.mean(mape))
print(np.mean(rmse))
print(np.mean(media))

456
456
12.1491717143906
0.5092387528969212
13.498319088751838
30.07565467449829


# Mismo dia del mes

In [143]:
df.FECHAHORA.min() + relativedelta(hours = 11, minutes = 40)

Timestamp('2019-04-01 00:00:00')

In [151]:
primera_fecha = df.FECHAHORA.min() + relativedelta(days = 30, hours = 11, minutes = 40)

daily_aqi = []

minute_aqi = 0
y_pred = {}

primer_index = df[df['FECHAHORA'] == primera_fecha].index[0]

index_1 = 0

minutos = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
bandera = 0

aqi_aux = 0

sumador = 0

for i in range(primer_index, df.index.stop, 288):
    j = i+288
    
    aqi_real = df.iloc[i:j, :]
    
    dia = aqi_real.DIA_SEM.value_counts().index[0]
    
    daily_aqi.append(np.asarray(aqi_real.AQI_MP2_5))
    
    aqi_7_dias = df.iloc[index_1:i, :]
    
    prediction_aqi = []
    
    for k in range(0, 24):
        
        for m in minutos:
            
            
            aqi_momento = aqi_7_dias[(aqi_7_dias['HORA'] == k) & (aqi_7_dias['MINUTO'] == m) & (aqi_7_dias['DIA_SEM'] == dia)]
            
            for idx in aqi_momento.index:
                
                aqi_aux = aqi_aux + aqi_momento.loc[idx, 'AQI_MP2_5']
                bandera = bandera + 1
                
                
            if bandera > 0:

                minute_aqi = aqi_aux / bandera
                #print(minute_aqi)
                
                aqi_aux = 0
                bandera = 0
                
                
                prediction_aqi.append(minute_aqi)
    
    
    y_pred[sumador] = prediction_aqi
    sumador = sumador + 1
    index_1 = index_1 + 288
    

print(len(daily_aqi))
print(len(y_pred))

mae = []
rmse = []
mape = []
media = []
for i in range(0, len(daily_aqi)-2):
    mae.append(mean_absolute_error(daily_aqi[i], y_pred[i]))
    mape.append(mean_absolute_percentage_error(daily_aqi[i], y_pred[i]))
    rmse.append(mean_squared_error(daily_aqi[i], y_pred[i], squared = False))
    media.append(daily_aqi[i].mean())
    
print(np.mean(mae))
print(np.mean(mape))
print(np.mean(rmse))
print(np.mean(media))

427
427
13.768764297385621
0.6303668516562771
14.766656474869034
28.943815359477124
